### Выгрузка из БК в ноутбук

In [1]:
!pip install pandas-gbq

    100% |████████████████████████████████| 92kB 1.2MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 8.1MB/s ta 0:00:011
    100% |████████████████████████████████| 71kB 4.6MB/s ta 0:00:01
    100% |████████████████████████████████| 1.1MB 3.1MB/s ta 0:00:011    98% |███████████████████████████████▌| 1.1MB 4.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for googleapis-common-protos ... done
  Stored in directory: /home/avarkentin/.cache/pip/wheels/da/6b/81/8573adcbe2aa2ecba92c341dfe19c5b5a733f4514297ba52b4
Successfully built googleapis-common-protos


In [6]:
import pandas as pd
from google.oauth2 import service_account

In [7]:
# Прописываем адрес к файлу с данными по сервисному аккаунту и получаем credentials для доступа к данным
credentials = service_account.Credentials.from_service_account_file(
    'ru-dataservices-ce567915256b.json')#ключ на компе от гугла

ТРЕНИРУЕМСЯ

In [6]:
# Формируем запрос и получаем количество вопросов с тегом "pandas", сгруппированные по дате создания
query = '''
SELECT DATE(creation_date) as date, COUNT(id) as questions
FROM
  [bigquery-public-data:stackoverflow.posts_questions]
WHERE tags LIKE '%pandas%'
GROUP BY
  date
'''

# Указываем идентификатор проекта
project_id = 'ru-dataservices'

In [7]:
# Выполняем запрос, записывая результат в dataframe
df = pd.read_gbq(query, project_id=project_id, credentials=credentials)

display(df.head(5))

,date,questions
0,2017-09-04,57
1,2013-05-26,7
2,2018-09-19,114
3,2018-09-02,54
4,2016-12-07,71


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2430 entries, 0 to 2429
Data columns (total 2 columns):
date         2430 non-null object
questions    2430 non-null int64
dtypes: int64(1), object(1)
memory usage: 38.0+ KB


БОЕВОЙ ЗАПРОС

In [34]:
query = '''
#standardSQL

SELECT 
event.a_afficheW
,event.a_c_gt_Banking
,event.a_c_gt_Real_estate
,event.a_c_gt_Good_deals
,event.a_c_gt_Furniture
,event.a_c_gt_Tennis
,event.a_c_gt_Building_and_civil_engineering
,event.a_c_gt_Entrepreneur
,event.a_c_gt_Family
,event.a_c_gt_Finance
,event.a_c_gt_Marriage_civil_union
,event.a_c_gt_Loans
,event.a_c_gt_Interior_design
,event.a_c_gt_Asset_management
,event.a_c_gt_Business_Administration_and_Management
,event.a_c_gt_Car_buyers
,event.a_c_gt_Careers_and_occupational_training
,event.a_c_gt_Catchup_TV
,event.a_c_gt_Weather
,event.a_c_gt_Winter_Holidays
,event.a_c_gt_Children
,event.a_c_gt_Soccer
,event.a_c_gt_Games_consoles
,event.a_c_gt_Cloud_computing
,event.a_c_gt_Churn_Telecom
,event.a_c_gt_Car_Rental
,event.a_c_gt_Clothing
,event.a_c_gt_Fruits_and_vegetables
,event.a_s_a_11_14
,event.a_s_a_15_17
,event.a_s_a_18_24
,event.a_s_a_25_34
,event.a_s_a_35_49
,event.a_s_a_50_64
,event.a_s_a_65_110
,event.a_s_g_m
,event.a_s_g_f
,event.a_s_s_i
,event.a_s_i_0
,event.a_s_i_1
,event.a_s_i_2
,event.a_s_i_3
,event.a_s_i_4
,event.a_latitude
,event.a_longitude
,event.a_s_e_po
,event.a_s_e_pr
,event.a_s_e_s
,event.a_s_e_u


FROM (
  SELECT ARRAY_AGG(t ORDER BY t.a_insert_time DESC LIMIT 1)
    [OFFSET(0)] event
  FROM 
    `ru-dataservices.BIGSEA_EU.RU_uu_2019*` t 
  WHERE _table_suffix BETWEEN '0101' AND '0125'
  
  GROUP BY a_afficheW
)
INNER JOIN
  --Uploaded cookies
  `ru-dataservices.result.ID_MIC_cookies` mic
ON
  --Key field
  event.a_afficheW = mic.string_field_0
ORDER BY 
  a_afficheW
LIMIT 1000
'''

# Указываем идентификатор проекта
project_id = 'ru-dataservices'

In [35]:
#установил диалект стандартный скл, чтоб не выдавал ошибку
df1 = pd.read_gbq(query, project_id=project_id, credentials=credentials, dialect = 'standard')

display(df1.head(5))

,a_afficheW,a_c_gt_Banking,a_c_gt_Real_estate,a_c_gt_Good_deals,a_c_gt_Furniture,a_c_gt_Tennis,a_c_gt_Building_and_civil_engineering,a_c_gt_Entrepreneur,a_c_gt_Family,a_c_gt_Finance,...,a_s_i_1,a_s_i_2,a_s_i_3,a_s_i_4,a_latitude,a_longitude,a_s_e_po,a_s_e_pr,a_s_e_s,a_s_e_u
0,084M98T3jaDT,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,9,13,52.7185,41.4266,NaN,13,20,19
1,1Qn4rkJ@EEHV,9.0,9.0,NaN,NaN,13.0,NaN,8.0,13.0,NaN,...,4,8,18,17,55.5913,37.7428,NaN,18,4,7
2,FNQcwgbenY4U,18.0,16.0,11.0,NaN,9.0,NaN,NaN,3.0,15.0,...,4,16,12,8,55.8602,37.5169,9.0,6,5,10
3,GIqveNtC9-8Y,19.0,19.0,12.0,19.0,2.0,NaN,1.0,NaN,11.0,...,19,2,18,18,55.7359,37.6330,1.0,16,17,1
4,JNOhAbovLlQk,17.0,NaN,NaN,13.0,NaN,NaN,19.0,9.0,7.0,...,8,8,10,5,55.7616,37.6411,1.0,7,16,16


Радуемся собственной охеренности

### Загрузка в БК с ноутбука

In [37]:
df.head()

,date,questions
0,2017-09-04,57
1,2013-05-26,7
2,2018-09-19,114
3,2018-09-02,54
4,2016-12-07,71


In [39]:
df.to_gbq('result.test_BQ', project_id=project_id, if_exists='fail', private_key='ru-dataservices-ce567915256b.json')

1it [00:08,  8.53s/it]


### Google Storage: загрузка и выгрузка

In [41]:
!pip install google-cloud-storage

    100% |████████████████████████████████| 61kB 733kB/s ta 0:00:011


In [4]:
#list of buckets
import os
from google.cloud import storage

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/avarkentin/Загрузки/ru-dataservices-ce567915256b.json"

# Instantiates a client
storage_client = storage.Client()

# List all the buckets available
for bucket in storage_client.list_buckets():
    print(bucket)

<Bucket: cohort_mr>
<Bucket: data_nespresso>
<Bucket: ikea_segments>
<Bucket: mic_development>
<Bucket: mvid>
<Bucket: nestle_dogs>
<Bucket: nestle_pro>
<Bucket: q4_unilever>
<Bucket: rc_dashboard>
<Bucket: tapaddata>
<Bucket: uaz>


In [68]:
#качаем на комп

# client = storage.Client(credentials=credentials, project='myproject')
bucket = storage_client.get_bucket('uaz')
blob = bucket.get_blob('uaz_clust_rich')
for i in bucket.list_blobs():
    print(i)

blob.download_to_filename('uaz_clust_rich2')#имя под которым сохраняем на комп, дата на компе файла будет та, что в хранилище

<Blob: uaz, patriot1_000000000000>
<Blob: uaz, patriot2_000000000000>
<Blob: uaz, patriot_1_decr.csv>
<Blob: uaz, patriot_2_decr.csv>
<Blob: uaz, pro_1_decr.csv>
<Blob: uaz, pro_1_rich>
<Blob: uaz, pro_2_decr.csv>
<Blob: uaz, pro_2_rich>
<Blob: uaz, uaz_clust_rich>


In [69]:
bucket = storage_client.get_bucket('uaz')
blob = bucket.blob('test2') #имя для хранилища
blob.upload_from_filename('uaz_clust_rich2')# имя файла на компе

for i in bucket.list_blobs():#список файлов в хранилище
    print(i)





<Blob: uaz, patriot1_000000000000>
<Blob: uaz, patriot2_000000000000>
<Blob: uaz, patriot_1_decr.csv>
<Blob: uaz, patriot_2_decr.csv>
<Blob: uaz, pro_1_decr.csv>
<Blob: uaz, pro_1_rich>
<Blob: uaz, pro_2_decr.csv>
<Blob: uaz, pro_2_rich>
<Blob: uaz, test2>
<Blob: uaz, uaz_clust_rich>


### Запрос в БК и передача результатов в ГКС

In [70]:
!pip install bigquery-python

    100% |████████████████████████████████| 61kB 1.5MB/s ta 0:00:011
    100% |████████████████████████████████| 225kB 6.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for bigquery-python ... done
  Stored in directory: /home/avarkentin/.cache/pip/wheels/f2/77/f1/af7646fa3cb56986c2b890cffcdd6511123d7e4a303eac979b
  Running setup.py bdist_wheel for httplib2 ... done
  Stored in directory: /home/avarkentin/.cache/pip/wheels/6d/41/4b/2b369d6e2b7eaebcdd423516d3fb659c7658c16a2be8fd04ec
Successfully built bigquery-python httplib2


In [72]:
!pip install --upgrade oauth2client

    100% |████████████████████████████████| 102kB 1.2MB/s a 0:00:011


In [2]:
from bigquery import get_client
from google.oauth2 import service_account
# BigQuery project id as listed in the Google Developers Console.
project_id = 'ru-dataservices'

# Service account email address as listed in the Google Developers Console.
service_account = 'service-test@ru-dataservices.iam.gserviceaccount.com'

# JSON key provided by Google
json_key = 'ru-dataservices-ce567915256b.json'
 
client = get_client(json_key_file=json_key, readonly = False)

In [ ]:
# write to permanent table with UDF in query string
from google.cloud import bigquery

external_udf_uris = ["gs://uaz/test3/*"]
query = '''
#standardSQL

SELECT 
event.a_afficheW
,event.a_c_gt_Banking
,event.a_c_gt_Real_estate
,event.a_c_gt_Good_deals
,event.a_c_gt_Furniture
,event.a_c_gt_Tennis
,event.a_c_gt_Building_and_civil_engineering
,event.a_c_gt_Entrepreneur
,event.a_c_gt_Family
,event.a_c_gt_Finance
,event.a_c_gt_Marriage_civil_union
,event.a_c_gt_Loans
,event.a_c_gt_Interior_design
,event.a_c_gt_Asset_management
,event.a_c_gt_Business_Administration_and_Management
,event.a_c_gt_Car_buyers
,event.a_c_gt_Careers_and_occupational_training
,event.a_c_gt_Catchup_TV
,event.a_c_gt_Weather
,event.a_c_gt_Winter_Holidays
,event.a_c_gt_Children
,event.a_c_gt_Soccer
,event.a_c_gt_Games_consoles
,event.a_c_gt_Cloud_computing
,event.a_c_gt_Churn_Telecom
,event.a_c_gt_Car_Rental
,event.a_c_gt_Clothing
,event.a_c_gt_Fruits_and_vegetables
,event.a_s_a_11_14
,event.a_s_a_15_17
,event.a_s_a_18_24
,event.a_s_a_25_34
,event.a_s_a_35_49
,event.a_s_a_50_64
,event.a_s_a_65_110
,event.a_s_g_m
,event.a_s_g_f
,event.a_s_s_i
,event.a_s_i_0
,event.a_s_i_1
,event.a_s_i_2
,event.a_s_i_3
,event.a_s_i_4
,event.a_latitude
,event.a_longitude
,event.a_s_e_po
,event.a_s_e_pr
,event.a_s_e_s
,event.a_s_e_u


FROM (
  SELECT ARRAY_AGG(t LIMIT 1)
    [OFFSET(0)] event
  FROM 
    `ru-dataservices.BIGSEA_EU.RU_uu_2019*` t 
  WHERE _table_suffix BETWEEN '0101' AND '0125'
  
  GROUP BY a_afficheW
)
INNER JOIN
  --Uploaded cookies
  `ru-dataservices.result.ID_MIC_cookies` mic
ON
  --Key field
  event.a_afficheW = mic.string_field_0
ORDER BY 
  a_afficheW
LIMIT 1000
'''
# Set use_legacy_sql to True to use legacy SQL syntax.
# job_config = bigquery.QueryJobConfig()
# job_config.use_legacy_sql = False

# job = client.write_to_table(
#   query,
#   table = 'testtable',
#   external_udf_uris=external_udf_uris,
#   use_legacy_sql = False)


# job = client.export_data_to_uris(
#   query,
#   table = 'test table',
#   destination_uris=external_udf_uris,
#   use_legacy_sql = False)
# try:
#     job_resource = client.wait_for_job(job, timeout=60)
#     print (job_resource)
# except BigQueryTimeoutException:
#     print ("Timeout")

In [113]:
# #  Submit an async query
# job_id, _results = client.query(query, external_udf_uris=external_udf_uris, use_legacy_sql = False)
# # Poll for query completion.
# complete, row_count = client.check_job(job_id)

# # Retrieve the results.
# if complete:
#     results = client.get_query_rows(job_id)

In [116]:
# complete

False

#### Создание таблицы в БК после запроса

In [127]:
from google.cloud import bigquery
client = bigquery.Client()
dataset_id = 'result'

job_config = bigquery.QueryJobConfig()
job_config.use_legacy_sql = False
# Set the destination table
table_ref = client.dataset(dataset_id).table('test_BQ2')
job_config.destination = table_ref
sql = query

# Start the query, passing in the extra configuration.
query_job = client.query(
    sql,
    # Location must match that of the dataset(s) referenced in the query
    # and of the destination table.
    job_config=job_config)  # API request - starts the query

query_job.result()  # Waits for the query to finish
print('Query results loaded to table {}'.format(table_ref.path))

Query results loaded to table /projects/ru-dataservices/datasets/result/tables/test_BQ2


#### Импорт таблицы из БК в ГКС

In [8]:
from google.cloud import bigquery
client = bigquery.Client()
bucket_name = 'ikea_segments'
project = 'ru-dataservices'
dataset_id = 'result'
table_id = 'ikea_pot_1_rich'

destination_uri = 'gs://{}/{}'.format(bucket_name, 'ikea_pot_1_rich.csv')
dataset_ref = client.dataset(dataset_id, project=project)
table_ref = dataset_ref.table(table_id)

extract_job = client.extract_table(
    table_ref,
    destination_uri)
    # Location must match that of the source table.)  # API request
extract_job.result()  # Waits for job to complete.

print('Exported {}:{}.{} to {}'.format(
    project, dataset_id, table_id, destination_uri))

NotFound: 404 Not found: Table ru-dataservices:result.ikea_pot_1_rich was not found in location EU